In [1]:
import torch
from meu_dataset import MeuDataset,avaliar_descritor,calcular_matching
from teste_util import *

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
path_dataset = "./data/datasets/features_path_flowers_dataset.pt"
# Carregar o dataset do arquivo "meu_dataset.pt"
meu_dataset = MeuDataset.load_from_file(path_dataset)
#verificar se o objeto meu dataset está retornando o tensor correto
assert isinstance(meu_dataset,MeuDataset), 'o tipo de retorno não é MeuDataset'
assert isinstance(meu_dataset[0][0],torch.Tensor), 'o tipo de retorno não é torch.Tensor'


In [2]:
import gc
from torch.optim.lr_scheduler import ExponentialLR
gc.collect()
torch.cuda.empty_cache()
batch_size_siam = 50
from torch.utils.data import random_split, DataLoader


train_dataset, val_dataset, test_dataset = random_split(meu_dataset, [0.5,0.3,0.2])

# Crie uma instância do DataLoader usando seu conjunto de dados personalizado
dataloader_train = DataLoader(train_dataset, batch_size=batch_size_siam, shuffle=False)
dataloader_val = DataLoader(val_dataset, batch_size=batch_size_siam, shuffle=False)
dataloader_test = DataLoader(test_dataset, batch_size=batch_size_siam, shuffle=False)

In [3]:
from torch.nn.functional import pairwise_distance
from tqdm import tqdm

def loss_fn(output_pos, output_neg, margin=15,total=100): 
    acertos_pos = torch.sum(output_pos[:, 0] == output_pos[:, 1])
    false_pos = torch.sum(output_pos[:, 0] != output_pos[:, 1])
    false_neg = output_neg.shape[0]
    total_erros = false_pos + false_neg
    
    loss = (total-acertos_pos+total_erros).to(torch.float32).to(output_pos.device).detach().requires_grad_(True)
    loss = torch.relu(loss)  # Aplicar ReLU para descartar os valores negativos
    return loss/total

# def loss_fn(output_pos, output_neg, margin=15,total=100): 
#     acertos_pos = torch.sum(output_pos[:, 0] == output_pos[:, 1])
#     false_pos = torch.sum(output_pos[:, 0] != output_pos[:, 1])
#     false_neg = output_neg.shape[0]
#     total_erros = false_pos + false_neg
    
#     loss = (total-acertos_pos+total_erros).to(torch.float32).to(output_pos.device).detach().requires_grad_(True)
#     loss = torch.relu(loss)  # Aplicar ReLU para descartar os valores negativos
#     return loss/total

#Create methods to train the model
def train_one_epoch(model, data_loader, optimizer, loss_fn, device='cpu', is_training=True):
    model.train(is_training)
    total_loss = 0.
    # Definir os intervalos de colunas
    
    progress_bar = tqdm(data_loader)
    for idx,data in enumerate(progress_bar):
        #extrair as features e orientações
        batch_in,batch_out = data[0].to(device),data[1].to(device)        
        descs_original = model(batch_in)# encontrando descritores
        descs_transform = model(batch_out)# encontrando descritores
        match_pos,match_neg  = calcular_matching(descs_original, descs_transform,th=1.00)# calculando os matches
        loss = loss_fn(match_pos,match_neg ,margin=(batch_in.size(0)*3/4),total = batch_in.size(0)*4)
        
        if is_training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        total_loss += loss.item()
        progress_bar.set_description(f'Loss: {loss.item()} - Total Loss: {total_loss}')
    return total_loss

In [4]:
from e2cnn import gspaces
from e2cnn import nn as enn    #the equivariant layer we need to build the model
from torch import nn
import torch

class Feature(nn.Module):
    def __init__(self,n_channel=2) -> None:
        super().__init__()
        r2_act = gspaces.Rot2dOnR2(N=18)

        feat_type_in  = enn.FieldType(r2_act,  n_channel*[r2_act.trivial_repr])
        feat_type_out = enn.FieldType(r2_act, 2*n_channel*[r2_act.regular_repr])
        self.input_type = feat_type_in

        self.block1 = enn.SequentialModule(
                enn.R2Conv(feat_type_in, feat_type_out, kernel_size=3, padding=0, bias=False),
                enn.InnerBatchNorm(feat_type_out),
                enn.ReLU(feat_type_out, inplace=True)
                )

        self.pool1 = enn.PointwiseAvgPoolAntialiased(feat_type_out, sigma=0.66, stride=1, padding=0)

        feat_type_in  = self.block1.out_type
        feat_type_out = enn.FieldType(r2_act,  4*n_channel*[r2_act.regular_repr])
        self.block2 = enn.SequentialModule(
                enn.R2Conv(feat_type_in, feat_type_out, kernel_size=3, padding=0, bias=False),
                enn.InnerBatchNorm(feat_type_out),
                enn.ReLU(feat_type_out, inplace=True),
                )
        # self.pool2 = enn.PointwiseAvgPool(feat_type_out, 21)

        feat_type_in  = feat_type_out
        feat_type_out = enn.FieldType(r2_act,  8*n_channel*[r2_act.regular_repr])
        self.block3 = enn.SequentialModule(
                enn.R2Conv(feat_type_in, feat_type_out, kernel_size=3, padding=0, bias=False),
                enn.InnerBatchNorm(feat_type_out),
                enn.ReLU(feat_type_out, inplace=True),
                enn.GroupPooling(feat_type_out),
                )
        self.pool = enn.PointwiseAdaptiveAvgPool(self.block3.out_type,1)

    def forward(self,X1)->torch.Tensor:
        x = enn.GeometricTensor(X1, self.input_type)
        n_dim = X1.shape[-1]
        mask = enn.MaskModule(self.input_type, n_dim, margin=2).to(X1.device)
        x = mask(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.pool(x)
        x = x.tensor.reshape(x.shape[0],-1)
        return x

n_channel =8
model =Feature(n_channel=n_channel).to(device)

/home/wagner/.local/lib/python3.11/site-packages/e2cnn/nn/modules/r2_conv/basisexpansion_singleblock.py:80: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at ../aten/src/ATen/native/IndexingUtils.h:27.)
  full_mask[mask] = norms.to(torch.uint8)


In [5]:
gc.collect()
torch.cuda.empty_cache()



def train(model,dataloader_train,dataloader_val):
    optimizer_siamese = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
    scheduler = ExponentialLR(optimizer_siamese, gamma=0.75)
    best_loss = float('inf')
    best_model = None
    epochs_without_improvement = 0
    epochs = 50
    patience = 10
    
    for epoch in range(epochs):
        train_loss = train_one_epoch(model,dataloader_train , optimizer_siamese, loss_fn, device=device, is_training=True)
        with torch.no_grad():
            test_loss = train_one_epoch(model,dataloader_val , optimizer_siamese, loss_fn, device=device, is_training=False)

        # Verificar se a perda melhorou
        if test_loss < best_loss:
            best_loss = test_loss
            epochs_without_improvement = 0
            best_model = model.state_dict()        
        else:
            epochs_without_improvement += 1
        
        if (epoch % 3 == 0) and (epoch != 0):
            scheduler.step()
            save_model(model, PATH_MODEL)
        
        # Verificar a condição de parada
        if epochs_without_improvement == patience:
            print(f"No improvement in loss for {epochs_without_improvement} epochs. Training stopped.")
            break

        print(f"Epoch [{epoch}/{epochs}] - Running Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Initial LR: {optimizer_siamese.param_groups[0]['initial_lr']:.6f}, Current LR: {optimizer_siamese.param_groups[0]['lr']:.6f}, Epochs without Improvement: {epochs_without_improvement}")

    # Carregar a melhor configuração do modelo
    model.load_state_dict(best_model)
    print(f'Epoch: {epoch}, Best Loss: {best_loss:.4f}')
    return model

PATH_MODEL = './data/models/feature_flowers_sp.pt'
model = train(model,dataloader_train,dataloader_val)

Loss: 1.0 - Total Loss: 154.4299976825714: 100%|██████████| 152/152 [00:07<00:00, 20.37it/s]                
Loss: 1.0277777910232544 - Total Loss: 92.78777611255646: 100%|██████████| 91/91 [00:03<00:00, 23.78it/s] 


Epoch [0/50] - Running Loss: 154.4300, Test Loss: 92.7878, Initial LR: 0.001000, Current LR: 0.001000, Epochs without Improvement: 0


Loss: 1.0 - Total Loss: 154.4299976825714: 100%|██████████| 152/152 [00:07<00:00, 21.55it/s]                
Loss: 1.0277777910232544 - Total Loss: 92.78777611255646: 100%|██████████| 91/91 [00:03<00:00, 23.67it/s] 


Epoch [1/50] - Running Loss: 154.4300, Test Loss: 92.7878, Initial LR: 0.001000, Current LR: 0.001000, Epochs without Improvement: 1


Loss: 1.0 - Total Loss: 154.4299976825714: 100%|██████████| 152/152 [00:07<00:00, 21.53it/s]                
Loss: 1.0277777910232544 - Total Loss: 92.78777611255646: 100%|██████████| 91/91 [00:03<00:00, 23.54it/s] 


...

Epoch [8/50] - Running Loss: 154.4300, Test Loss: 92.7878, Initial LR: 0.001000, Current LR: 0.000563, Epochs without Improvement: 8


Loss: 1.0 - Total Loss: 154.4299976825714: 100%|██████████| 152/152 [00:07<00:00, 21.38it/s]                
Loss: 1.0277777910232544 - Total Loss: 92.78777611255646: 100%|██████████| 91/91 [00:03<00:00, 23.55it/s] 


Model saved to ./data/models/feature_flowers_sp.pt
Epoch [9/50] - Running Loss: 154.4300, Test Loss: 92.7878, Initial LR: 0.001000, Current LR: 0.000422, Epochs without Improvement: 9


Loss: 1.0 - Total Loss: 154.4299976825714: 100%|██████████| 152/152 [00:07<00:00, 21.22it/s]                
Loss: 1.0277777910232544 - Total Loss: 92.78777611255646: 100%|██████████| 91/91 [00:03<00:00, 23.35it/s] 

No improvement in loss for 10 epochs. Training stopped.
Epoch: 10, Best Loss: 92.7878


In [6]:
save_model(model, PATH_MODEL)
model =Feature(n_channel=n_channel)
model.to(device)
load_model(model, PATH_MODEL,device)

Model saved to ./data/models/feature_flowers_sp.pt
Model loaded from ./data/models/feature_flowers_sp.pt


In [7]:
model =model.eval()
with torch.no_grad():
    total_acertos,total_erros,total_elementos = avaliar_descritor(dataloader_test, model,th=0.35)
sub_conjunto = total_elementos//2
print(f'Total de elementos no DataLoader: {total_elementos}')
print(f'Acertei: {total_acertos}/{sub_conjunto} Errei: {total_erros}/{sub_conjunto}')

100%|██████████| 61/61 [00:02<00:00, 24.25it/s]

Total de elementos no DataLoader: 12096
Acertei: 5386/6048 Errei: 301/6048


### Refazer o treinamento para fazer o descritor na imagem original ao inves da feature

In [8]:
path_dataset = "./data/datasets/img_path_flowers_dataset.pt"
meu_dataset = MeuDataset.load_from_file(path_dataset)
train_dataset, val_dataset, test_dataset = random_split(meu_dataset, [0.4,0.4,0.2])

# Crie uma instância do DataLoader usando seu conjunto de dados personalizado
dataloader_train = DataLoader(train_dataset, batch_size=batch_size_siam, shuffle=True)
dataloader_val = DataLoader(val_dataset, batch_size=batch_size_siam, shuffle=True)
dataloader_test = DataLoader(test_dataset, batch_size=batch_size_siam, shuffle=True)

In [9]:
n_channel =1
model =Feature(n_channel=n_channel).to(device)
PATH_MODEL = './data/models/img_flowers_sp.pt'
model = train(model,dataloader_train,dataloader_val)

Loss: 1.0729167461395264 - Total Loss: 131.3229147195816: 100%|██████████| 121/121 [00:02<00:00, 42.20it/s] 
Loss: 1.0833333730697632 - Total Loss: 130.63332951068878: 100%|██████████| 121/121 [00:02<00:00, 48.60it/s]


Epoch [0/50] - Running Loss: 131.3229, Test Loss: 130.6333, Initial LR: 0.001000, Current LR: 0.001000, Epochs without Improvement: 0


Loss: 1.0833333730697632 - Total Loss: 131.96333050727844: 100%|██████████| 121/121 [00:02<00:00, 40.81it/s]
Loss: 1.0729167461395264 - Total Loss: 129.93291223049164: 100%|██████████| 121/121 [00:02<00:00, 45.70it/s]


Epoch [1/50] - Running Loss: 131.9633, Test Loss: 129.9329, Initial LR: 0.001000, Current LR: 0.001000, Epochs without Improvement: 0


Loss: 1.0729167461395264 - Total Loss: 131.77291321754456: 100%|██████████| 121/121 [00:02<00:00, 41.71it/s]
Loss: 1.0729167461395264 - Total Loss: 130.7129133939743: 100%|██████████| 121/121 [00:02<00:00, 49.79it/s] 


...

Model saved to ./data/models/img_flowers_sp.pt
Epoch [9/50] - Running Loss: 131.6850, Test Loss: 129.9629, Initial LR: 0.001000, Current LR: 0.000422, Epochs without Improvement: 8


Loss: 1.0625 - Total Loss: 130.44249629974365: 100%|██████████| 121/121 [00:02<00:00, 47.59it/s]            


Epoch [10/50] - Running Loss: 131.7125, Test Loss: 130.4425, Initial LR: 0.001000, Current LR: 0.000422, Epochs without Improvement: 9


Loss: 1.0416667461395264 - Total Loss: 131.79166388511658: 100%|██████████| 121/121 [00:02<00:00, 40.39it/s]
Loss: 1.0729167461395264 - Total Loss: 130.7629133462906: 100%|██████████| 121/121 [00:02<00:00, 46.58it/s] 

No improvement in loss for 10 epochs. Training stopped.
Epoch: 11, Best Loss: 129.9329


In [10]:
model =model.eval()
with torch.no_grad():
    total_acertos,total_erros,total_elementos = avaliar_descritor(dataloader_test, model,th=0.25)
sub_conjunto = total_elementos//2
print(f'Total de elementos no DataLoader: {total_elementos}')
print(f'Acertei: {total_acertos}/{sub_conjunto} Errei: {total_erros}/{sub_conjunto}')
save_model(model, PATH_MODEL)

100%|██████████| 61/61 [00:01<00:00, 51.41it/s]

Total de elementos no DataLoader: 12096
Acertei: 4589/6048 Errei: 883/6048
Model saved to ./data/models/img_flowers_sp.pt
